In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.microsoft import EdgeChromiumDriverManager
import time
import random

service = Service(EdgeChromiumDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

driver = webdriver.Edge(service=service, options=options)

query = ["2024", "2023", "2022", "2021", "2020", "2019", "2018", "2017", "2016", "2015"]
max_pages = 100
titles_list = []

for i in range(query):
    for page in range(max_pages):
        start = page * 10  # Google Scholar menampilkan 10 artikel per halaman
        search_url = f"https://scholar.google.com/scholar?q={query[i]}&start={start}"
        
        driver.get(search_url)

        # Gunakan delay acak agar lebih manusiawi
        time.sleep(random.uniform(2, 4))  # Rentang waktu 2-4 detik

        # Deteksi CAPTCHA
        if "I'm not a robot" in driver.page_source or "Tolong tunjukkan bahwa Anda bukan robot" in driver.page_source:
            print("CAPTCHA terdeteksi! Silakan selesaikan secara manual.")
            while True:
                time.sleep(1)  # Cek setiap 1 detik untuk mempercepat deteksi
                if "I'm not a robot" not in driver.page_source and "Tolong tunjukkan bahwa Anda bukan robot" not in driver.page_source:
                    print("CAPTCHA selesai, lanjutkan scraping.")
                    break

        # Tunggu elemen artikel muncul
        wait = WebDriverWait(driver, 10)
        try:
            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "h3.gs_rt")))
        except:
            print(f"Elemen artikel tidak ditemukan di halaman {page + 1}, kemungkinan masih ada CAPTCHA.")
            continue  # Lewati halaman ini dan lanjutkan ke halaman berikutnya

        # Ambil semua judul artikel
        titles = driver.find_elements(By.CSS_SELECTOR, "h3.gs_rt")
        if not titles:
            print(f"Tidak ada judul di halaman {page + 1}, mungkin Google memblokir akses.")
            break

        for title in titles:
            if title.text.strip():
                titles_list.append(title.text.strip())

        print(f"Halaman {page + 1} selesai, total artikel terkumpul: {len(titles_list)}")

# Tutup browser setelah selesai
driver.quit()


: 

In [ ]:
import pandas as pd

# Simpan data ke dalam DataFrame
df = pd.DataFrame({"Judul": titles_list})

# Simpan ke file CSV
df.to_csv("judul_scholar.csv", index=False, encoding="utf-8")

print("Data berhasil disimpan ke judul_scholar.csv")


In [ ]:
df.head()